In [1]:
import pandas as pd
import numpy as np
import glob
import random
import math

from math import sin, cos, radians

def rotate_point(point, angle, center_point=(0, 0)):
    """Rotates a point around center_point(origin by default)
    Angle is in degrees.
    Rotation is counter-clockwise
    """
    angle_rad = radians(angle % 360)
    # Shift the point so that center_point becomes the origin
    new_point = (point[0] - center_point[0], point[1] - center_point[1])
    new_point = (new_point[0] * cos(angle_rad) - new_point[1] * sin(angle_rad),
                 new_point[0] * sin(angle_rad) + new_point[1] * cos(angle_rad))
    # Reverse the shifting we have done
    new_point = (new_point[0] + center_point[0], new_point[1] + center_point[1])
    return new_point

In [2]:
data = pd.read_csv("traj_train_set.csv")
sequence_length = 200                      ## set sequence length of one trajectory
total_timesteps = data['timestamp'].count()


In [5]:
df_list = []

x_seq_o = [] 
y_seq_o = [] 

x_rel_seq_o = []
y_rel_seq_o = []
track_seq_o = []

for indx in range(0, total_timesteps, sequence_length):
#     print("THE INDEX: ", indx)
    traj = data.loc[indx:indx+sequence_length-1]
    traj = traj.sort_values("timestamp", ascending=True)
    start_x = traj.head(1)['longitude'].item()
    start_y = traj.head(1)['latitude'].item()

    x_seq = [] 
    y_seq = [] 

    x_rel_seq = []
    y_rel_seq = []
    track_seq = []

    x_seq = [i for i in traj['longitude']]
    y_seq = [i for i in traj['latitude']]
    
    for i, row in traj.iterrows():
        x_rel_seq.append(row['longitude'] - start_x)
        y_rel_seq.append(row['latitude'] - start_y)
        track_seq.append(int(indx))

    if (y_rel_seq[-1] != 0 and x_rel_seq[-1] != 0):
        x_seq_o = x_seq_o + x_seq
        y_seq_o = y_seq_o + y_seq

        x_rel_seq_o = x_rel_seq_o + x_rel_seq
        y_rel_seq_o = y_rel_seq_o + y_rel_seq
        track_seq_o = track_seq_o + track_seq
dict_pd = {'Vehicle_ID': track_seq_o, 'X': x_seq_o, 'Y': y_seq_o, 'X_REL': x_rel_seq_o, 'Y_REL': y_rel_seq_o} 
df = pd.DataFrame(dict_pd)
df_list.append(df)


C:\Users\daavi\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: `item` has been deprecated and will be removed in a future version
  
C:\Users\daavi\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: `item` has been deprecated and will be removed in a future version


In [6]:
data_combined = pd.concat(df_list, axis=0, ignore_index=True)

In [7]:
unique_vehicle = data_combined['Vehicle_ID'].unique()
countr = 0
for vehicle in unique_vehicle:
    countr += 1
    if (countr % 2 == 0):
        rad = random.randrange(360)
        start_x = data_combined['X'][data_combined['Vehicle_ID'] == vehicle].head(1).item()
        start_y = data_combined['Y'][data_combined['Vehicle_ID'] == vehicle].head(1).item()
        rotated = data_combined[['X', 'Y']][data_combined['Vehicle_ID'] == vehicle].apply(lambda row: rotate_point([row['X'],row['Y']], rad, [start_x, start_y]),axis=1, result_type="expand") 
        data_combined['X'][data_combined['Vehicle_ID'] == vehicle] = rotated[0]
        data_combined['Y'][data_combined['Vehicle_ID'] == vehicle] = rotated[1]
        start_x = data_combined['X'][data_combined['Vehicle_ID'] == vehicle].head(1).item()
        start_y = data_combined['Y'][data_combined['Vehicle_ID'] == vehicle].head(1).item()
        data_combined['X_REL'][data_combined['Vehicle_ID'] == vehicle] = data_combined['X'][data_combined['Vehicle_ID'] == vehicle] - start_x
        data_combined['Y_REL'][data_combined['Vehicle_ID'] == vehicle] = data_combined['Y'][data_combined['Vehicle_ID'] == vehicle] - start_y

C:\Users\daavi\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: `item` has been deprecated and will be removed in a future version
  import sys
C:\Users\daavi\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: `item` has been deprecated and will be removed in a future version
  
C:\Users\daavi\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\daavi\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\U

In [8]:
data_combined.to_csv("traj_train.csv")